In [1]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../"))

# from desc import set_device
# set_device("cpu")

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.constants import mu_0
import plotly.graph_objects as go
import warnings

from desc.backend import jnp
from desc.transform import Transform
from desc.plotting import plot_1d,plot_3d,plot_comparison, plot_surfaces, plot_grid
from desc.optimize import Optimizer
from desc.examples import get
from desc.objectives import get_fixed_boundary_constraints, maybe_add_self_consistency
from desc.grid import LinearGrid, QuadratureGrid, Grid

from desc.compute.utils import _compute as compute_fun
from desc.nestor import Nestor
from desc.objectives.objective_funs import _Objective
from desc.singularities import (
    DFTInterpolator,
    FFTInterpolator,
    virtual_casing_biot_savart,
    _nonsingular_part,
    _singular_part,
    kernels,
)
from desc.utils import Timer, errorif, warnif

from desc.objectives import ObjectiveFunction, BaroundIslandError

DESC version 0.11.1+571.g6790b22c6.dirty,using JAX backend, jax version=0.4.25, jaxlib version=0.4.25, dtype=float64
Using device: CPU, with 9.43 GB available memory


In [7]:
eq = get("precise_QA")
# eq.pressure = 0
# eq.solve();
objective = ObjectiveFunction(BaroundIslandError(eq=eq, rho=0.5))
objective.build(verbose=3)

Building objective: B consistency error
Precomputing transforms
Timer: Precomputing transforms = 65.3 ms
Timer: Objective build = 241 ms


In [ ]:
from desc.magnetic_fields._core import B_from_surface_integral
import time

eq = get("precise_QA")

t0 = time.time()
grid_source = LinearGrid(rho=1, M=550, N=550, NFP=int(eq.NFP))
grid_eval = LinearGrid(rho=0.5, M=4, N=0, NFP=int(eq.NFP))

keys = [
    "K_vc",
    "|e_theta x e_zeta|",
    "R",
    "phi",
    "Z",
]
data_source = eq.compute(keys, grid=grid_source)
data_eval = eq.compute(keys, grid=grid_eval)

re = jnp.vstack((data_eval["R"], data_eval["phi"], data_eval["Z"])).T
rs = jnp.vstack((data_source["R"], data_source["phi"], data_source["Z"])).T
dA = grid_source.weights * data_source["|e_theta x e_zeta|"] / grid_source.NFP
K = data_source["K_vc"]
print(f"Setting parameters took: {time.time()-t0:.2f} seconds")

Bplasma2 = B_from_surface_integral(re, rs, K, eq.NFP, dA)

scale = 1
fig,ax = plot_surfaces(eq,theta=0,rho=np.arange(0.1,1.01,0.1), phi=0.0,figsize=(8,8),lcfs_color="k")
ax[0].quiver(data_eval["R"],data_eval["Z"],data_eval["B"][:,0],data_eval["B"][:,2],label=r"$B_{eval-grid}$",scale=scale,color="r")
ax[0].quiver(data_eval["R"],data_eval["Z"],Bplasma2[:,0],Bplasma2[:,2],label=r"$B_{surface-integral}$",scale=scale,color="g")
ax[0].legend(fontsize=12)
fig=plt.gcf()

In [ ]:
from desc.magnetic_fields import field_line_integrate, ToroidalMagneticField
from desc.magnetic_fields._core import biot_savart_general
from desc.compute import rpz2xyz, rpz2xyz_vec, xyz2rpz_vec
from desc.backend import jnp, fori_loop

def B_from_K_trace(re, params=None, basis="rpz",source_grid=grid_source):
    phi_re = re[:, 1]
    re = rpz2xyz(re)
    def nfp_loop(j, f):
        # calculate (by rotating) rs, rs_t, rz_t
        phi = (grid_source.nodes[:, 2] + j * 2 * jnp.pi / grid_source.NFP) % (
            2 * jnp.pi
        )
        # new coords are just old R,Z at a new phi (bc of discrete NFP symmetry)
        rs = jnp.vstack((_rs[:, 0], phi, _rs[:, 2])).T
        rs = rpz2xyz(rs)
        
        K = rpz2xyz_vec(_K, phi=phi)
        fj = biot_savart_general(
            re,
            rs,
            K,
            _dV,
        )
        f += fj
        return f

    Bplasma = fori_loop(0, grid_source.NFP, nfp_loop, jnp.zeros_like(re))
    Bplasma = xyz2rpz_vec(Bplasma, phi=phi_re)
    return Bplasma

grid_source = LinearGrid(rho=1, M=550, N=550, NFP=int(eq.NFP))

keys = [
    "K_vc",
    "|e_theta x e_zeta|",
    "R",
    "phi",
    "Z",
]
data_source = eq.compute(keys, grid=grid_source)
_rs = jnp.vstack((data_source["R"], data_source["phi"], data_source["Z"])).T
_dV = grid_source.weights * data_source["|e_theta x e_zeta|"] / grid_source.NFP
_K = data_source["K_vc"]

ntransit=50
n_R_points = 10

phis = np.arange(0, ntransit * 2 * np.pi, 2 * np.pi)
rrr = np.linspace(1.17,1.25,n_R_points) # initial R positions of field-lines to trace
Bfield = ToroidalMagneticField(1,1)
Bfield.compute_magnetic_field = B_from_K_trace
field_R, field_Z = field_line_integrate(rrr, np.zeros_like(rrr), phis, Bfield, source_grid=grid_source)